In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

**导入数据**

In [ ]:
train = pd.read_csv(r"../input/bike-sharing-demand/train.csv", parse_dates = ["datetime"])
test = pd.read_csv(r"../input/bike-sharing-demand/test.csv", parse_dates = ["datetime"])

In [ ]:
test1 = pd.read_csv(r"../input/bike-sharing-demand/test.csv", parse_dates = ["datetime"])

把训练数据和测试数据按year、hour、dayofweek和month划分

In [ ]:
train["year"] = train["datetime"].dt.year
train["hour"] = train["datetime"].dt.hour
train["dayofweek"] = train["datetime"].dt.dayofweek
train["month"] = train["datetime"].dt.month

test["year"] = test["datetime"].dt.year
test["hour"] = test["datetime"].dt.hour
test["dayofweek"] = test["datetime"].dt.dayofweek
test["month"] = test["datetime"].dt.month

In [ ]:
cols = list(train.columns.values)
cols.pop(cols.index('count'))
train = train[cols + ['count']]

**查看训练数据的数据情况**

In [ ]:
train.head()

查看数据集的详细信息

In [ ]:
train.info()

通过柱形图查看数据分布count和season、weather和windspeed的关系特征

In [ ]:
fig, axes = plt.subplots(3,1, figsize = (20,12))

sns.countplot(train["season"], ax = axes[0], palette = "pastel")
sns.countplot(train["weather"], ax = axes[1], palette = "pastel")
sns.countplot(train["windspeed"], ax = axes[2])

查看数据分布count和atemp、temp和month的关系特征

In [ ]:
fig, axes = plt.subplots(3,1, figsize = (20,12))

sns.countplot(train["atemp"], ax = axes[0])
sns.countplot(train["temp"], ax = axes[1])
sns.countplot(train["month"], ax = axes[2])

In [ ]:
y1=train["count"]
y2=train["casual"]
y3=train["registered"]

对这三个数据进行取对数减小差异

In [ ]:
y1 = np.log1p(y1)
y2 = np.log1p(y2)
y3 = np.log1p(y3)

通过前边的特征分析，去除无用的数据方便模型拟合

In [ ]:
train.drop("datetime", inplace = True, axis = 1)       #datetime
train.drop("atemp", inplace = True, axis = 1)       #atemp
train.drop("windspeed", inplace = True, axis = 1)       #windspeed
train.drop("casual", inplace = True, axis = 1)       #casual
train.drop("registered", inplace = True, axis = 1)      #registered
train.drop("month", inplace = True, axis = 1)      #month
train.drop("count", inplace = True, axis = 1)      #count  

In [ ]:
test.drop("datetime", inplace = True, axis = 1)       #datetime
test.drop("atemp", inplace = True, axis = 1)       #atemp
test.drop("windspeed", inplace = True, axis = 1)       #windspeed
test.drop("month", inplace = True, axis = 1)      #month  

In [ ]:
test

使用LGBM来进行预测，分别对count、casual和registered进行预测，并对预测结果进行处理得到最终的预测结果

In [ ]:
import lightgbm as lgb
hyperparameters = { 'colsample_bytree': 0.805,  'learning_rate': 0.015,
                    'num_leaves': 70, 'reg_alpha': 0.854, 'reg_lambda': 0.271, 
                    'subsample': 0.323, 'n_estimators': 993}
model = lgb.LGBMRegressor(**hyperparameters)
model.fit(train, y1)
preds = model.predict(test)

hyperparameters = { 'colsample_bytree': 0.725,  'learning_rate': 0.013,
                    'num_leaves': 56, 'reg_alpha': 0.754, 'reg_lambda': 0.071, 
                    'subsample': 0.523, 'n_estimators': 1093}
model = lgb.LGBMRegressor(**hyperparameters)
model.fit(train, y2)
preds1 = model.predict(test)

hyperparameters = { 'colsample_bytree': 0.639,  'learning_rate': 0.011,
                    'num_leaves': 30, 'reg_alpha': 0.351, 'reg_lambda': 0.587,
                   'subsample': 0.916, 'n_estimators': 2166}
model = lgb.LGBMRegressor(**hyperparameters, )
model.fit(train, y3)
preds2 = model.predict(test)

对于三个预测结果按权重求和并写入结果文件

In [ ]:
submission=pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")
submission["count"] = 0.95*(np.expm1(preds1) + np.expm1(preds2)) + 0.05*np.expm1(preds)
submission.to_csv("answer.csv", index=False)

In [ ]:
submission